# Texts as Matrices

In [1]:
import numpy as np
import spacy
from collections import Counter
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
nlp = spacy.load('en')

In [2]:
def read_paragraphs(fname):
    with open(fname, 'r') as f:
        text = f.read()
    paragraphs = [p for p in text.split('\n\n') if len(p) > 0]
    return paragraphs

trump_par_texts = read_paragraphs('nss/trump_nss.txt')
obama_par_texts = read_paragraphs('nss/obama_nss.txt')
par_texts = trump_par_texts + obama_par_texts
k = len(trump_par_texts)
len(par_texts), len(trump_par_texts), len(obama_par_texts)

(550, 400, 150)

## Tokenization Formula
A common step to many text analysis algorithms is to first convert the raw text into sets of tokens. Spacy does most of the work here, there are just a few decisions that need to be made depending on the application: which tokens to include and how to represent the tokens as strings.

In [3]:
def parse_tok(tok):
    '''Convert spacy token object to string.'''
    number_ents = ('NUMBER','MONEY','PERCENT','QUANTITY','CARDINAL','ORDINAL')
    if tok.ent_type_ == '':
        return tok.text.lower()
    elif tok.ent_type_ in number_ents:
        return tok.ent_type_
    else:
        return tok.text
    
def use_tok(tok):
    '''Decide to use token or not.'''
    return tok.is_ascii and not tok.is_space and len(tok.text.strip()) > 0
    
def parse_doc(doc):
    # combine multi-word entities into their own tokens (just a formula)
    for ent in doc.ents:
        ent.merge(tag=ent.root.tag_, ent_type=ent.root.ent_type_)
    return [parse_tok(tok) for tok in doc if use_tok(tok)]

tokenized_pars = [parse_doc(par) for par in nlp.pipe(par_texts)]

In [4]:
# first paragraph, first five tokens
tokenized_pars[0][:5]

['an', 'America', 'that', 'is', 'safe']

## Bag-of-Words and Document-Term Matrices

In [5]:
min_tf = 10
vectorizer = CountVectorizer(tokenizer = lambda x: x, preprocessor=lambda x:x, min_df=min_tf)
corpus = vectorizer.fit_transform(tokenized_pars)
vocab = vectorizer.get_feature_names()
COOC = corpus.toarray()
len(vocab), type(corpus), corpus.shape, COOC.shape

(524, scipy.sparse.csr.csr_matrix, (550, 524), (550, 524))

Now reduce vocabulary to words which appear at least once in both corpora.

In [6]:
valid_cols = (COOC[:k].sum(axis=0) > 0) & (COOC[k:].sum(axis=0) > 0)
rm_wordids = np.argwhere(~valid_cols)[:,0]
vocab = [w for i,w in enumerate(vocab) if i not in rm_wordids]
COOC = COOC[:,valid_cols]
COOC.shape

(550, 507)

Now remove documents that have none of the selected vocab words.

In [59]:
min_wordcount = 10
zerosel = COOC.sum(axis=1) < min_wordcount
zeroind = np.argwhere(zerosel)[:,0]
tokenized_pars = [toks for i,toks in enumerate(tokenized_pars) if i not in zeroind]
par_texts = [par for i,par in enumerate(par_texts) if i not in zeroind]
k = k - (zeroind < k).sum()
COOC = COOC[~zerosel]

COOC.shape, k, len(tokenized_pars), len(par_texts)

((441, 507), 324, 441, 441)

In [60]:
print(vocab[:10])
COOC[:10,:10]

['(', ')', ',', '-', '.', ':', ';', 'Africa', 'America', 'American']


array([[ 0,  0,  6,  0,  3,  0,  0,  0,  4,  1],
       [ 0,  0,  3,  0,  1,  0,  0,  0,  1,  1],
       [ 0,  0,  6,  1,  4,  0,  0,  0,  0,  2],
       [ 0,  0,  8,  1,  4,  0,  0,  0,  0,  0],
       [ 0,  0,  4,  0,  2,  0,  0,  0,  0,  1],
       [ 0,  0,  5,  0,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  3,  0,  3,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  2,  0,  0,  0,  0,  1],
       [ 0,  0, 13,  0,  5,  0,  0,  0,  2,  0],
       [ 1,  1,  9,  1,  3,  0,  0,  0,  0,  1]])

In [61]:
(COOC.sum(axis=1)==0).sum()

0

Using bag of words, we now compare word distributions averaged across the documents from Trump and Obama. Here we present the words that are more likely to be in the Trump NSS compared to Obama's.

In [62]:
topn = 30
trump_cts, obama_cts = COOC[:k].sum(axis=0), COOC[k:].sum(axis=0)
logdiff = np.log( (trump_cts/trump_cts.sum()) / (obama_cts / obama_cts.sum()))
indices = logdiff.argsort()[-topn:][::-1]
print(', '.join(['{} ({:.2f})'.format(vocab[idx],logdiff[idx]) for idx in indices]))

: (3.03), compete (2.27), under (2.20), conditions (2.12), nation (2.08), seeks (2.03), liberty (2.03), industry (2.03), Americans (2.00), minded (1.94), continues (1.94), sovereign (1.94), adversaries (1.90), life (1.89), want (1.83), encourage (1.83), principles (1.71), technologies (1.67), practices (1.58), undermine (1.58), The United States (1.56), improve (1.56), vital (1.51), Today (1.43), companies (1.43), domestic (1.43), operate (1.43), operations (1.43), property (1.34), into (1.34)


### Topic Modeling
Because topic models are computed directly from document-term matrices, I demonstrate the use of both the NMF and LDA algorithms. After computing each model, I then compute the log ratio of probabilities of subcorpora being associated with each topic. Larger values mean Trump's documents are more closely associated with the topic while more negative values are more closely associated with Obama.

In [63]:
# non-negative matrix factorization (similar to pca but for only positive-entry matrices)
nmf_model = NMF(n_components=10).fit(COOC)
doc_topics = nmf_model.transform(COOC)
topic_words = nmf_model.components_
topic_words.shape, doc_topics.shape

((10, 507), (441, 10))

In [64]:
# for nmf compare distributions between sources
trump_av = doc_topics[:k].mean(axis=0)
obama_av = doc_topics[k:].mean(axis=0)
logratio = np.log(trump_av/obama_av)
logratio

array([-0.41283237, -0.63728651, -0.84966108, -0.74124851, -0.48178337,
       -0.45959257, -0.87741655, -0.15518991, -1.72093414, -0.42493658])

In [65]:
# non-negative matrix factorization (similar to pca but for only positive-entry matrices)
lda_model = LatentDirichletAllocation(n_components=10).fit(COOC)
doc_topics = lda_model.transform(COOC)
topic_words = lda_model.components_
topic_words.shape, doc_topics.shape

((10, 507), (441, 10))

In [66]:
# for nmf compare distributions between sources
trump_av = doc_topics[:k].mean(axis=0)
obama_av = doc_topics[k:].mean(axis=0)
logratio = np.log(trump_av/obama_av)
logratio

array([ 0.54148573,  0.64658564,  1.72492766,  1.28210013,  0.09256617,
       -0.68714229,  1.15892343, -0.50059194,  1.25416577,  1.66449761])

### Pointwise Mutual Information
The [pointwise mutual information](https://en.wikipedia.org/wiki/Pointwise_mutual_information) calculates the level of association between two variables, document and word in this case (as designated by the 2-dimensional distribution), by controlling for both the frequency of a word and the number of words in a document. Higher values mean that the word is more uniquely associated with the document statistically than not.

Positive pointwise mutual information is a variant of PMI which sets negative values (words which are less associated with documents than expected) to zero. While we loose some information here, this solves the problem of -infinity values caused by taking log(0) and has shown to still be a robust measure.
Levy, Goldberg, Dagan (2015) _Improving Distributional Similarity with Lessons Learned from Word Embeddings_ ([link])(https://levyomer.files.wordpress.com/2015/03/improving-distributional-similarity-tacl-2015.pdf).

In [67]:
import matrices # from included script

In [68]:
PPMI = matrices.calc_ppmi(COOC)
print(PPMI.shape)
PPMI[:5,:5]

(441, 507)


array([[0.        , 0.        , 5.59187069, 0.        , 5.22512691],
       [0.        , 0.        , 5.57527119, 0.        , 4.90442862],
       [0.        , 0.        , 5.39127814, 5.97436491, 5.24029597],
       [0.        , 0.        , 5.66166589, 6.0289911 , 5.29492211],
       [0.        , 0.        , 5.55210853, 0.        , 5.1853647 ]])

The power of PMI is that you can go back to the original documents to examine the words most closely associated with them compared to all other docs in the corpus.

In [69]:
target_docid = 0
temp_PPMI = PPMI.copy()
for i in range(5):
    idx = temp_PPMI[target_docid,:].argmax()
    print('{} ({:.2f})'.format(vocab[idx], temp_PPMI[target_docid,idx]), end=' ')
    temp_PPMI[target_docid,idx] = 0
print('\n', par_texts[target_docid])

uphold (9.22) foundation (9.12) liberty (8.88) enduring (8.81) confidence (8.79) 
 An America that is safe, prosperous, and free at home is an America with the strength, confidence, and will to lead abroad. It is an America that can preserve peace, uphold liberty , and create enduring advantages for the American people. Putting America first is the duty of our government and the foundation for U.S. leadership in the world.


### Singular Value Decomposition of PPMI Matrix
This is simply decomposing the PPMI matrix into singular values, which we used to compress the matrix. Note that in cases where your vocab is larger than the number of documents, read the [implementation notes](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html#numpy.linalg.svd).

In [101]:
SVD = matrices.calc_svd(PPMI,100)
SVD = SVD - SVD.mean(axis=0) # center
SVD = SVD / np.linalg.norm(SVD, axis=1)[:,np.newaxis] # normalize
svd_vars = SVD.var(axis=0)/SVD.var(axis=0).sum()
svd_vars # variance explained by each dimension (if arg two of calc_svd is None), need to normalize by all eigenvalues

array([0.02406945, 0.03615592, 0.02495261, 0.02607884, 0.02111817,
       0.02161532, 0.0193136 , 0.0166996 , 0.01554702, 0.01459172,
       0.01496909, 0.01428053, 0.01504344, 0.01366774, 0.01302898,
       0.01325838, 0.01327273, 0.01362083, 0.01144731, 0.01272128,
       0.01213789, 0.01192576, 0.01205809, 0.01139212, 0.0105516 ,
       0.01079131, 0.01092207, 0.0110332 , 0.0108154 , 0.010916  ,
       0.01073725, 0.00979404, 0.01038939, 0.01052787, 0.0102035 ,
       0.00924232, 0.01018133, 0.0102287 , 0.00971629, 0.00972708,
       0.00938486, 0.00887874, 0.00909769, 0.00933416, 0.00913616,
       0.00886873, 0.00891708, 0.00894251, 0.00803768, 0.00816864,
       0.00833853, 0.00833357, 0.00808068, 0.0081198 , 0.00811789,
       0.00829316, 0.00790994, 0.00762248, 0.00740023, 0.00774648,
       0.00736724, 0.00800482, 0.00715861, 0.00721603, 0.007624  ,
       0.00712471, 0.00747347, 0.00732539, 0.00737229, 0.00773569,
       0.0069016 , 0.00722822, 0.00685813, 0.00736594, 0.00663

We can now examine these SVD representations as vectors in an embedding space: these are essentially document (paragraph in this case) embeddings. Now we can examine how different documents are distributed across the space.

In [107]:
# calcualte average veector norm and norm of averagae vector
np.linalg.norm(SVD, axis=1).mean(), np.linalg.norm(SVD.mean(axis=0))

(1.0, 0.041524799018364346)

### Document Embeddings
Because we have document embeddings, we can measure the field that situates each document. In this case, we'll identify the document closest to the mean vector (center of the field), and then the document furthest from the mean.

In [117]:
#dists = np.linalg.norm(SVD - SVD.mean(axis=0), axis=1)
#dists = SVD.dot(SVD.mean(axis=0))
dists = SVD.dot(SVD.T).sum(axis=1)
par_lens = np.array([len(toks) for toks in tokenized_pars])
print('correlation of word freq with distance:', np.corrcoef(par_lens, dists)[0,1])
print('norm of mean:', np.linalg.norm(SVD.mean(axis=0)))
sortind = dists.argsort() # first is greatest dist
dists[sortind][:5], dists[sortind[::-1]][:5]

correlation of word freq with distance: -0.8267501433369303
norm of mean: 0.041524799018364346


(array([-9.36939204, -7.84807113, -7.31467506, -6.98291822, -6.88569154]),
 array([10.61537976,  9.94522212,  9.22464922,  8.87196866,  8.86831118]))

In [115]:
# get closest to center of field
for i in range(2,0,-1):
    ind = sortind[i]
    d = np.linalg.norm(SVD[ind] - SVD.mean(axis=0))
    print('doc', ind, 'd =', d)
    print(par_texts[ind], end='\n\n')

doc 332 d = 1.0172990899760197
This strategy builds on the progress of the last 6 years, in which our active leadership has helped the world recover from a global economic crisis and respond to an array of emerging challenges. Our progress includes strengthening an unrivaled alliance system, underpinned by our enduring partnership with Europe, while investing in nascent multilateral forums like the G-20 and East Asia Summit. We brought most of our troops home after more than a decade of honorable service in two wars while adapting our counterterrorism strategy for an evolving terrorist threat. We led a multinational coalition to support the Afghan government to take responsibility for the security of their country, while supporting Afghanistans first peaceful, democratic transition of power. The United States led the international response to natural disasters, including the earthquake in Haiti, the earthquake and tsunami in Japan, and the typhoon in the Philippines to save lives, prev

In [116]:
# get closest to center of field
for i in range(2):
    ind = sortind[::-1][i]
    d = np.linalg.norm(SVD[ind] - SVD.mean(axis=0))
    print('doc', ind, 'd =', d)
    print(par_texts[ind], end='\n\n')

doc 28 d = 0.976515231358157
Protect the American People, the Homeland, and the American Way of Life

doc 191 d = 0.9780701717108031
IMPROVE ATTRIBUTION, ACCOUNTABILITY, AND RESPONSE:  We will invest in capabilities to support and improve our ability to attribute cyberattacks, to allow for rapid response.



## Token Co-Occurrence Matrices

In [22]:
trump_cooc = (corpus[:k].T*corpus[:k]).toarray()
obama_cooc = (corpus[k:].T*corpus[k:]).toarray()
trump_cooc.shape, obama_cooc.shape

((524, 524), (524, 524))

In [23]:
# compute vocab from words that appear at least once in each subcorpora
min_count = 1
trump_tok_cts = Counter([tok for doc in trump_pars for tok in doc])
obama_tok_cts = Counter([tok for doc in obama_pars for tok in doc])
words = set(trump_tok_cts.keys()) & set(obama_tok_cts.keys())
word_cts = {w:(obama_tok_cts[w]+trump_tok_cts[w]) for w in words}
vocab = list(sorted(word_cts.items(), key=lambda x: x[0], reverse=True))
n = len(vocab)
n

NameError: name 'trump_pars' is not defined

## Word Co-Occurrence Matrix
One common technique used in corpus linguistics and other social sciences is the co-occurrence matrix. This model of the text is implicitly used in word2vec algorithms. It can be defined in a number of ways, but here we will define it as the number of times two words appear in the same paragraph together.

In [ ]:
def calc_cooccur(docs):
    cooccur = dict()
    for doc in docs:
        for i,w1 in enumerate(doc):
            for j,w2 in enumerate(doc[i:]):
                k = (w1,w2)
                if k not in cooccur:
                    cooccur[k] = 0
                cooccur[k] += 1
    return cooccur
trump_cooc_dict = calc_cooccur(trump_pars)
obama_cooc_dict = calc_cooccur(obama_pars)
print(list(trump_cooc_dict.keys())[:4])

In [ ]:
def cooc_matrix(cooc_dict, vocab):
    voc_index = {tok:i for i,tok in enumerate(vocab)}
    cooc_mat = np.zeros((n,n))
    for toks,ct in cooc_dict.items():
        w1,w2 = toks
        cooc_mat[voc_index[w1],voc_index[w2]] = ct
    return cooc_mat
trump_cooc = cooc_matrix(trump_cooc_dict, vocab)
obama_cooc = cooc_matrix(trump_cooc_dict, vocab)